In [2]:
import os
import pandas as pd
import json
import xml.etree.ElementTree as ET

In [ ]:
# folder_path = './all_bills'
# json_folder = './json'

# bill_data = []

# # counter = 0

# def parse_bill(xml_file):
#     parsed_data = {}

#     try:
#         tree = ET.parse(xml_file)
#         root = tree.getroot()

#         bill = root.find('bill')

#         if bill is not None:
#             parsed_data['number'] = bill.find('number').text if bill.find('number') is not None else None
#             parsed_data['title'] = bill.find('title').text if bill.find('title') is not None else None
#             parsed_data['introducedDate'] = bill.find('introducedDate').text if bill.find('introducedDate') is not None else None
#             parsed_data['updateDate'] = bill.find('updateDate').text if bill.find('updateDate') is not None else None
#             parsed_data['originChamber'] = bill.find('originChamber').text if bill.find('originChamber') is not None else None
#             parsed_data['type'] = bill.find('type').text if bill.find('type') is not None else None
#             parsed_data['congress'] = bill.find('congress').text if bill.find('congress') is not None else None

#             actions = bill.find('actions')

#             for item in actions.findall('item'):
#                 parsed_data['actionCode'] = item.find('actionCode').text if item.find('actionCode') is not None else None   
#                 break
            
#             if actions is not None:
#                 first_item = actions.find('item')
#                 if first_item is not None:
#                     text = first_item.find('text').text if first_item.find('text') is not None else None
#                     parsed_data['lastResult'] = text
#                 else:
#                     print(f"No 'item' element found in {xml_file}")
#         else:
#             print(f"No 'bill' element found in {xml_file}")
#     except Exception as e:
#         print(f"Error processing {xml_file}")
    
#     return parsed_data

# for file in os.listdir(folder_path):
#     if file.endswith('.xml'):
#         file_path = os.path.join(folder_path, file)
        
#         bill_data.append(parse_bill(file_path))

#         # counter += 1

#         # if counter >= 5:
#         #     break

# df = pd.DataFrame(bill_data)

In [3]:
df = pd.read_csv('smaller_set.csv')

In [313]:
df

,number,title,introducedDate,updateDate,originChamber,type,congress,actionCode,lastResult,final,updateTime,duration
0,1,Regarding consent to assemble outside the seat...,2003-01-07,2023-01-15,House,HCONRES,108,0,Message on Senate action sent to the House.,0,21:04:32,7313
1,10,Condemning the Democratic People's Republic of...,2003-01-08,2023-01-15,House,HCONRES,108,H11100,Referred to the House Committee on Internation...,0,21:04:33,7312
2,100,Recognizing the 100th anniversary year of the ...,2003-03-18,2023-01-15,House,HCONRES,108,0,"Referred to the Subcommittee on Commerce, Trad...",0,22:18:40,7243
3,101,Expressing the sense of the Congress that Publ...,2003-03-19,2023-01-15,House,HCONRES,108,H11100,Referred to the House Committee on Internation...,0,22:18:40,7242
4,102,Expressing the sense of Congress that Congress...,2003-03-19,2023-01-15,House,HCONRES,108,H11100,Referred to the House Committee on Internation...,0,22:18:40,7242
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,435,Youth PROMISE Act,2009-02-13,2022-02-03,Senate,S,111,0,Read twice and referred to the Committee on th...,0,06:07:15,4738
49996,436,SAFETY Act,2009-02-13,2022-01-12,Senate,S,111,0,Read twice and referred to the Committee on th...,0,14:37:55,4716
49997,437,A bill to amend the Internal Revenue Code of 1...,2009-02-13,2019-11-15,Senate,S,111,0,Read twice and referred to the Committee on Fi...,0,21:55:02,3927
49998,438,Improved Medical Decision Incentive Act of 2009,2009-02-13,2019-11-15,Senate,S,111,0,Read twice and referred to the Committee on Fi...,0,21:55:02,3927


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   number          50000 non-null  int64 
 1   title           50000 non-null  object
 2   introducedDate  50000 non-null  object
 3   updateDate      50000 non-null  object
 4   originChamber   50000 non-null  object
 5   type            50000 non-null  object
 6   congress        50000 non-null  int64 
 7   actionCode      50000 non-null  object
 8   lastResult      50000 non-null  object
 9   final           50000 non-null  int64 
 10  updateTime      50000 non-null  object
 11  duration        50000 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 4.6+ MB


In [5]:
df['actionCode'].fillna(0, inplace=True)
df = df.dropna()
df

C:\Users\megku\AppData\Local\Temp\ipykernel_30096\877540312.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['actionCode'].fillna(0, inplace=True)


,number,title,introducedDate,updateDate,originChamber,type,congress,actionCode,lastResult,final,updateTime,duration
0,1,Regarding consent to assemble outside the seat...,2003-01-07,2023-01-15,House,HCONRES,108,0,Message on Senate action sent to the House.,0,21:04:32,7313
1,10,Condemning the Democratic People's Republic of...,2003-01-08,2023-01-15,House,HCONRES,108,H11100,Referred to the House Committee on Internation...,0,21:04:33,7312
2,100,Recognizing the 100th anniversary year of the ...,2003-03-18,2023-01-15,House,HCONRES,108,0,"Referred to the Subcommittee on Commerce, Trad...",0,22:18:40,7243
3,101,Expressing the sense of the Congress that Publ...,2003-03-19,2023-01-15,House,HCONRES,108,H11100,Referred to the House Committee on Internation...,0,22:18:40,7242
4,102,Expressing the sense of Congress that Congress...,2003-03-19,2023-01-15,House,HCONRES,108,H11100,Referred to the House Committee on Internation...,0,22:18:40,7242
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,435,Youth PROMISE Act,2009-02-13,2022-02-03,Senate,S,111,0,Read twice and referred to the Committee on th...,0,06:07:15,4738
49996,436,SAFETY Act,2009-02-13,2022-01-12,Senate,S,111,0,Read twice and referred to the Committee on th...,0,14:37:55,4716
49997,437,A bill to amend the Internal Revenue Code of 1...,2009-02-13,2019-11-15,Senate,S,111,0,Read twice and referred to the Committee on Fi...,0,21:55:02,3927
49998,438,Improved Medical Decision Incentive Act of 2009,2009-02-13,2019-11-15,Senate,S,111,0,Read twice and referred to the Committee on Fi...,0,21:55:02,3927


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   number          50000 non-null  int64 
 1   title           50000 non-null  object
 2   introducedDate  50000 non-null  object
 3   updateDate      50000 non-null  object
 4   originChamber   50000 non-null  object
 5   type            50000 non-null  object
 6   congress        50000 non-null  int64 
 7   actionCode      50000 non-null  object
 8   lastResult      50000 non-null  object
 9   final           50000 non-null  int64 
 10  updateTime      50000 non-null  object
 11  duration        50000 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 4.6+ MB


In [7]:
df['actionCode'] = df['actionCode'].astype(str)
df['final'] = df['actionCode'].apply(lambda x: x in ['E30000', 'E40000', '36000'])

df

,number,title,introducedDate,updateDate,originChamber,type,congress,actionCode,lastResult,final,updateTime,duration
0,1,Regarding consent to assemble outside the seat...,2003-01-07,2023-01-15,House,HCONRES,108,0,Message on Senate action sent to the House.,False,21:04:32,7313
1,10,Condemning the Democratic People's Republic of...,2003-01-08,2023-01-15,House,HCONRES,108,H11100,Referred to the House Committee on Internation...,False,21:04:33,7312
2,100,Recognizing the 100th anniversary year of the ...,2003-03-18,2023-01-15,House,HCONRES,108,0,"Referred to the Subcommittee on Commerce, Trad...",False,22:18:40,7243
3,101,Expressing the sense of the Congress that Publ...,2003-03-19,2023-01-15,House,HCONRES,108,H11100,Referred to the House Committee on Internation...,False,22:18:40,7242
4,102,Expressing the sense of Congress that Congress...,2003-03-19,2023-01-15,House,HCONRES,108,H11100,Referred to the House Committee on Internation...,False,22:18:40,7242
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,435,Youth PROMISE Act,2009-02-13,2022-02-03,Senate,S,111,0,Read twice and referred to the Committee on th...,False,06:07:15,4738
49996,436,SAFETY Act,2009-02-13,2022-01-12,Senate,S,111,0,Read twice and referred to the Committee on th...,False,14:37:55,4716
49997,437,A bill to amend the Internal Revenue Code of 1...,2009-02-13,2019-11-15,Senate,S,111,0,Read twice and referred to the Committee on Fi...,False,21:55:02,3927
49998,438,Improved Medical Decision Incentive Act of 2009,2009-02-13,2019-11-15,Senate,S,111,0,Read twice and referred to the Committee on Fi...,False,21:55:02,3927


In [8]:
df.head()

,number,title,introducedDate,updateDate,originChamber,type,congress,actionCode,lastResult,final,updateTime,duration
0,1,Regarding consent to assemble outside the seat...,2003-01-07,2023-01-15,House,HCONRES,108,0,Message on Senate action sent to the House.,False,21:04:32,7313
1,10,Condemning the Democratic People's Republic of...,2003-01-08,2023-01-15,House,HCONRES,108,H11100,Referred to the House Committee on Internation...,False,21:04:33,7312
2,100,Recognizing the 100th anniversary year of the ...,2003-03-18,2023-01-15,House,HCONRES,108,0,"Referred to the Subcommittee on Commerce, Trad...",False,22:18:40,7243
3,101,Expressing the sense of the Congress that Publ...,2003-03-19,2023-01-15,House,HCONRES,108,H11100,Referred to the House Committee on Internation...,False,22:18:40,7242
4,102,Expressing the sense of Congress that Congress...,2003-03-19,2023-01-15,House,HCONRES,108,H11100,Referred to the House Committee on Internation...,False,22:18:40,7242


In [9]:
df['introducedDate'] = pd.to_datetime(df['introducedDate'])
df['updateDate'] = pd.to_datetime(df['updateDate'])
df['updateDate'] = df['updateDate'].dt.date
df['introducedDate'] = df['introducedDate'].dt.date
df

,number,title,introducedDate,updateDate,originChamber,type,congress,actionCode,lastResult,final,updateTime,duration
0,1,Regarding consent to assemble outside the seat...,2003-01-07,2023-01-15,House,HCONRES,108,0,Message on Senate action sent to the House.,False,21:04:32,7313
1,10,Condemning the Democratic People's Republic of...,2003-01-08,2023-01-15,House,HCONRES,108,H11100,Referred to the House Committee on Internation...,False,21:04:33,7312
2,100,Recognizing the 100th anniversary year of the ...,2003-03-18,2023-01-15,House,HCONRES,108,0,"Referred to the Subcommittee on Commerce, Trad...",False,22:18:40,7243
3,101,Expressing the sense of the Congress that Publ...,2003-03-19,2023-01-15,House,HCONRES,108,H11100,Referred to the House Committee on Internation...,False,22:18:40,7242
4,102,Expressing the sense of Congress that Congress...,2003-03-19,2023-01-15,House,HCONRES,108,H11100,Referred to the House Committee on Internation...,False,22:18:40,7242
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,435,Youth PROMISE Act,2009-02-13,2022-02-03,Senate,S,111,0,Read twice and referred to the Committee on th...,False,06:07:15,4738
49996,436,SAFETY Act,2009-02-13,2022-01-12,Senate,S,111,0,Read twice and referred to the Committee on th...,False,14:37:55,4716
49997,437,A bill to amend the Internal Revenue Code of 1...,2009-02-13,2019-11-15,Senate,S,111,0,Read twice and referred to the Committee on Fi...,False,21:55:02,3927
49998,438,Improved Medical Decision Incentive Act of 2009,2009-02-13,2019-11-15,Senate,S,111,0,Read twice and referred to the Committee on Fi...,False,21:55:02,3927


In [10]:
df['updateDate'] = pd.to_datetime(df['updateDate'])
df['introducedDate'] = pd.to_datetime(df['introducedDate'])

df['duration'] = (df['updateDate'] - df['introducedDate']).dt.days
df

,number,title,introducedDate,updateDate,originChamber,type,congress,actionCode,lastResult,final,updateTime,duration
0,1,Regarding consent to assemble outside the seat...,2003-01-07,2023-01-15,House,HCONRES,108,0,Message on Senate action sent to the House.,False,21:04:32,7313
1,10,Condemning the Democratic People's Republic of...,2003-01-08,2023-01-15,House,HCONRES,108,H11100,Referred to the House Committee on Internation...,False,21:04:33,7312
2,100,Recognizing the 100th anniversary year of the ...,2003-03-18,2023-01-15,House,HCONRES,108,0,"Referred to the Subcommittee on Commerce, Trad...",False,22:18:40,7243
3,101,Expressing the sense of the Congress that Publ...,2003-03-19,2023-01-15,House,HCONRES,108,H11100,Referred to the House Committee on Internation...,False,22:18:40,7242
4,102,Expressing the sense of Congress that Congress...,2003-03-19,2023-01-15,House,HCONRES,108,H11100,Referred to the House Committee on Internation...,False,22:18:40,7242
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,435,Youth PROMISE Act,2009-02-13,2022-02-03,Senate,S,111,0,Read twice and referred to the Committee on th...,False,06:07:15,4738
49996,436,SAFETY Act,2009-02-13,2022-01-12,Senate,S,111,0,Read twice and referred to the Committee on th...,False,14:37:55,4716
49997,437,A bill to amend the Internal Revenue Code of 1...,2009-02-13,2019-11-15,Senate,S,111,0,Read twice and referred to the Committee on Fi...,False,21:55:02,3927
49998,438,Improved Medical Decision Incentive Act of 2009,2009-02-13,2019-11-15,Senate,S,111,0,Read twice and referred to the Committee on Fi...,False,21:55:02,3927


In [11]:
df['final'] = df['final'].astype(int)

In [322]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   number          50000 non-null  int64         
 1   title           50000 non-null  object        
 2   introducedDate  50000 non-null  datetime64[ns]
 3   updateDate      50000 non-null  datetime64[ns]
 4   originChamber   50000 non-null  object        
 5   type            50000 non-null  object        
 6   congress        50000 non-null  int64         
 7   actionCode      50000 non-null  object        
 8   lastResult      50000 non-null  object        
 9   final           50000 non-null  int32         
 10  updateTime      50000 non-null  object        
 11  duration        50000 non-null  int64         
dtypes: datetime64[ns](2), int32(1), int64(3), object(6)
memory usage: 4.4+ MB


In [ ]:
# df = pd.read_csv('filtered_bills.csv')

In [12]:
df_encoded = pd.get_dummies(df, columns=['type', 'originChamber'])
df = df_encoded
df

,number,title,introducedDate,updateDate,congress,actionCode,lastResult,final,updateTime,duration,type_HCONRES,type_HJRES,type_HR,type_HRES,type_S,type_SCONRES,type_SJRES,type_SRES,originChamber_House,originChamber_Senate
0,1,Regarding consent to assemble outside the seat...,2003-01-07,2023-01-15,108,0,Message on Senate action sent to the House.,0,21:04:32,7313,True,False,False,False,False,False,False,False,True,False
1,10,Condemning the Democratic People's Republic of...,2003-01-08,2023-01-15,108,H11100,Referred to the House Committee on Internation...,0,21:04:33,7312,True,False,False,False,False,False,False,False,True,False
2,100,Recognizing the 100th anniversary year of the ...,2003-03-18,2023-01-15,108,0,"Referred to the Subcommittee on Commerce, Trad...",0,22:18:40,7243,True,False,False,False,False,False,False,False,True,False
3,101,Expressing the sense of the Congress that Publ...,2003-03-19,2023-01-15,108,H11100,Referred to the House Committee on Internation...,0,22:18:40,7242,True,False,False,False,False,False,False,False,True,False
4,102,Expressing the sense of Congress that Congress...,2003-03-19,2023-01-15,108,H11100,Referred to the House Committee on Internation...,0,22:18:40,7242,True,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,435,Youth PROMISE Act,2009-02-13,2022-02-03,111,0,Read twice and referred to the Committee on th...,0,06:07:15,4738,False,False,False,False,True,False,False,False,False,True
49996,436,SAFETY Act,2009-02-13,2022-01-12,111,0,Read twice and referred to the Committee on th...,0,14:37:55,4716,False,False,False,False,True,False,False,False,False,True
49997,437,A bill to amend the Internal Revenue Code of 1...,2009-02-13,2019-11-15,111,0,Read twice and referred to the Committee on Fi...,0,21:55:02,3927,False,False,False,False,True,False,False,False,False,True
49998,438,Improved Medical Decision Incentive Act of 2009,2009-02-13,2019-11-15,111,0,Read twice and referred to the Committee on Fi...,0,21:55:02,3927,False,False,False,False,True,False,False,False,False,True


In [281]:
# df.to_csv('filtered_bills.csv', index=False)

In [14]:
df['final'] = df['final'].astype(int)

In [20]:
import warnings
import pickle
warnings.filterwarnings("ignore", category=Warning)
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error
from sklearn.model_selection import cross_val_score

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [21]:
bool_columns = df.select_dtypes(include='bool').columns
df[bool_columns] = df[bool_columns].astype(int)
df['introducedDate'] = df['introducedDate'].astype(str)
df['updateDate'] = df['updateDate'].astype(str)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   number                50000 non-null  int64 
 1   title                 50000 non-null  object
 2   introducedDate        50000 non-null  object
 3   updateDate            50000 non-null  object
 4   congress              50000 non-null  int64 
 5   actionCode            50000 non-null  object
 6   lastResult            50000 non-null  object
 7   final                 50000 non-null  int32 
 8   updateTime            50000 non-null  object
 9   duration              50000 non-null  int64 
 10  type_HCONRES          50000 non-null  int32 
 11  type_HJRES            50000 non-null  int32 
 12  type_HR               50000 non-null  int32 
 13  type_HRES             50000 non-null  int32 
 14  type_S                50000 non-null  int32 
 15  type_SCONRES          50000 non-null

In [23]:
#oversampling data
X = df.drop(columns=['title', 'introducedDate', 'updateDate', 'actionCode', 'lastResult', 'updateTime', 'final'])
y = df['final']

smote = SMOTE(sampling_strategy='minority')
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# model = LogisticRegression()
# model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# y_pred = model.predict(X_test)
# print(f"MAE: {mean_absolute_error(y_test, y_pred)}")
# print(f"MSE: {mean_squared_error(y_test, y_pred)}")
# print(f"RMSE: {root_mean_squared_error(y_test, y_pred)}")
# print(f"R² Score: {r2_score(y_test, y_pred)}")

# print(f"accuracy score: {accuracy_score(y_test, y_pred)}")
# print(f"precision score: {precision_score(y_test, y_pred)}")
# print(f"recall score: {recall_score(y_test, y_pred)}")
# print(f"f1 score: {f1_score(y_test, y_pred)}")
# print(f"confusion matrix: {confusion_matrix(y_test, y_pred)}")

MAE: 0.34538194264377947
MSE: 0.34538194264377947
RMSE: 0.5876920474566416
R² Score: -0.3815278040140915
accuracy score: 0.6546180573562205
precision score: 0.6500956022944551
recall score: 0.669916001244426
f1 score: 0.6598569969356486
confusion matrix: [[6163 3477]
 [3183 6460]]


In [32]:
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

probabilities = model.predict_proba(X_test)
pass_probabilities = probabilities[:, 1]

for i, prob in enumerate(pass_probabilities):
    print(f"Probability of passing = {prob:.2%}")

Probability of passing = 0.00%
Probability of passing = 63.83%
Probability of passing = 1.00%
Probability of passing = 100.00%
Probability of passing = 99.00%
Probability of passing = 0.00%
Probability of passing = 98.00%
Probability of passing = 0.00%
Probability of passing = 100.00%
Probability of passing = 0.00%
Probability of passing = 98.19%
Probability of passing = 0.00%
Probability of passing = 100.00%
Probability of passing = 0.00%
Probability of passing = 0.00%
Probability of passing = 81.25%
Probability of passing = 8.00%
Probability of passing = 100.00%
Probability of passing = 0.00%
Probability of passing = 0.00%
Probability of passing = 94.99%
Probability of passing = 0.00%
Probability of passing = 100.00%
Probability of passing = 100.00%
Probability of passing = 98.00%
Probability of passing = 100.00%
Probability of passing = 100.00%
Probability of passing = 75.33%
Probability of passing = 0.00%
Probability of passing = 100.00%
Probability of passing = 0.00%
Probability o

In [33]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [34]:
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

In [35]:
print(type(model))

<class 'sklearn.ensemble._forest.RandomForestClassifier'>


In [340]:
y_pred = model.predict(X_test)
print(f"MAE: {mean_absolute_error(y_test, y_pred)}")
print(f"MSE: {mean_squared_error(y_test, y_pred)}")
print(f"RMSE: {root_mean_squared_error(y_test, y_pred)}")
print(f"R² Score: {r2_score(y_test, y_pred)}")

print(f"accuracy score: {accuracy_score(y_test, y_pred)}")
print(f"precision score: {precision_score(y_test, y_pred)}")
print(f"recall score: {recall_score(y_test, y_pred)}")
print(f"f1 score: {f1_score(y_test, y_pred)}")
print(f"confusion matrix: {confusion_matrix(y_test, y_pred)}")

MAE: 0.052740756106414974
MSE: 0.052740756106414974
RMSE: 0.2296535567031675
R² Score: 0.78903287633354
accuracy score: 0.9472592438935851
precision score: 0.9395476214626229
recall score: 0.9565262288310615
f1 score: 0.9479609067185182
confusion matrix: [[9003  596]
 [ 421 9263]]


In [341]:
#using random forest to predict
fake_data = {
    'number': [101], 
    'congress': [118], 
    'duration': [45],  
    'type_HCONRES': [1],  
    'type_HJRES': [0],  
    'type_HR': [0],      
    'type_HRES': [0],    
    'type_S': [0],       
    'type_SCONRES': [0], 
    'type_SJRES': [0],   
    'type_SRES': [0],     
    'originChamber_House': [1],  
    'originChamber_Senate': [0]  
}

fake_data_df = pd.DataFrame(fake_data)

predicted_outcome = model.predict(fake_data_df)

print("Will this bill pass?")

if predicted_outcome[0] == 1:
    print("Yes")
else:
    print("No")

Will this bill pass?
No


In [342]:
import joblib

In [343]:
joblib.dump(model, 'model.pkl')

['model.pkl']